<a href="https://colab.research.google.com/github/kunalrdeshmukh/SRGAN/blob/master/SRGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


[Single Image Super-resolution using GAN](https://arxiv.org/abs/1609.04802)

In [1]:
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
import torch

import os

import torch
import torchvision
from torch import nn , optim
from torchvision import datasets, transforms
from torch.autograd import Variable

import torch.nn.functional as F

!pip install Pillow==4.0.0
!pip install PIL
!pip install image
from PIL import Image

tcmalloc: large alloc 1073750016 bytes == 0x57e08000 @  0x7fe92d8962a4 0x591a07 0x5b5d56 0x502e9a 0x506859 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x504c28 0x502540 0x502f3d 0x507641
    100% |████████████████████████████████| 5.6MB 5.8MB/s 
torchvision 0.2.1 has requirement pillow>=4.1.1, but you'll have pillow 4.0.0 which is incompatible.
  Found existing installation: Pillow 5.3.0
    Uninstalling Pillow-5.3.0:
      Successfully uninstalled Pillow-5.3.0
  Could not find a version that satisfies the requirement PIL (from versions: )
No matching distribution found for PIL
    100% |████████████████████████████████| 7.3MB 6.4MB/s 


In [2]:
img_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5,  0.5 , 0.5))
])

trainset = datasets.STL10(root='./data', split='train',download=True, transform=img_transform)

testset = datasets.STL10(root='./data', split='test',download=True, transform=img_transform)


Files already downloaded and verified


In [0]:
# code block to text image loader
# print(trainloader)

### Parameters

In [0]:
CHANNELS = 3
HEIGHT = 50
WIDTH = 50
LR_IMAGE_SIZE = 48
EPOCHS = 1
LOG_INTERVAL = 500
CUDA = torch.cuda.is_available()
BATCH_SIZE = 8
UPSCALING_FACTOR = 2

In [0]:
import math
irange = range


def make_grid(tensor, nrow=8, padding=2,
              normalize=False, range=None, scale_each=False, pad_value=0):
    """Make a grid of images.
    Args:
        tensor (Tensor or list): 4D mini-batch Tensor of shape (B x C x H x W)
            or a list of images all of the same size.
        nrow (int, optional): Number of images displayed in each row of the grid.
            The Final grid size is (B / nrow, nrow). Default is 8.
        padding (int, optional): amount of padding. Default is 2.
        normalize (bool, optional): If True, shift the image to the range (0, 1),
            by subtracting the minimum and dividing by the maximum pixel value.
        range (tuple, optional): tuple (min, max) where min and max are numbers,
            then these numbers are used to normalize the image. By default, min and max
            are computed from the tensor.
        scale_each (bool, optional): If True, scale each image in the batch of
            images separately rather than the (min, max) over all images.
        pad_value (float, optional): Value for the padded pixels.
    Example:
        See this notebook `here <https://gist.github.com/anonymous/bf16430f7750c023141c562f3e9f2a91>`_
    """
    if not (torch.is_tensor(tensor) or
            (isinstance(tensor, list) and all(torch.is_tensor(t) for t in tensor))):
        raise TypeError('tensor or list of tensors expected, got {}'.format(type(tensor)))

    # if list of tensors, convert to a 4D mini-batch Tensor
    if isinstance(tensor, list):
        tensor = torch.stack(tensor, dim=0)

    if tensor.dim() == 2:  # single image H x W
        tensor = tensor.view(1, tensor.size(0), tensor.size(1))
    if tensor.dim() == 3:  # single image
        if tensor.size(0) == 1:  # if single-channel, convert to 3-channel
            tensor = torch.cat((tensor, tensor, tensor), 0)
        tensor = tensor.view(1, tensor.size(0), tensor.size(1), tensor.size(2))

    if tensor.dim() == 4 and tensor.size(1) == 1:  # single-channel images
        tensor = torch.cat((tensor, tensor, tensor), 1)

    if normalize is True:
        tensor = tensor.clone()  # avoid modifying tensor in-place
        if range is not None:
            assert isinstance(range, tuple), \
                "range has to be a tuple (min, max) if specified. min and max are numbers"

        def norm_ip(img, min, max):
            img.clamp_(min=min, max=max)
            img.add_(-min).div_(max - min + 1e-5)

        def norm_range(t, range):
            if range is not None:
                norm_ip(t, range[0], range[1])
            else:
                norm_ip(t, float(t.min()), float(t.max()))

        if scale_each is True:
            for t in tensor:  # loop over mini-batch dimension
                norm_range(t, range)
        else:
            norm_range(tensor, range)

    if tensor.size(0) == 1:
        return tensor.squeeze()

    # make the mini-batch of images into a grid
    nmaps = tensor.size(0)
    xmaps = min(nrow, nmaps)
    ymaps = int(math.ceil(float(nmaps) / xmaps))
    height, width = int(tensor.size(2) + padding), int(tensor.size(3) + padding)
    grid = tensor.new(3, height * ymaps + padding, width * xmaps + padding).fill_(pad_value)
    k = 0
    for y in irange(ymaps):
        for x in irange(xmaps):
            if k >= nmaps:
                break
            grid.narrow(1, y * height + padding, height - padding)\
                .narrow(2, x * width + padding, width - padding)\
                .copy_(tensor[k])
            k = k + 1
    return grid


def save_image(tensor, filename, nrow=8, padding=2,
               normalize=False, range=None, scale_each=False, pad_value=0):
    """Save a given Tensor into an image file.
    Args:
        tensor (Tensor or list): Image to be saved. If given a mini-batch tensor,
            saves the tensor as a grid of images by calling ``make_grid``.
        **kwargs: Other arguments are documented in ``make_grid``.
    """
    from PIL import Image
    grid = make_grid(tensor, nrow=nrow, padding=padding, pad_value=pad_value,
                     normalize=normalize, range=range, scale_each=scale_each)
    ndarr = grid.mul(255).clamp(0, 255).byte().permute(1, 2, 0).cpu().numpy()
    im = Image.fromarray(ndarr)
    im.save(filename)

### Support functions

In [0]:
def sigmoid_mul(x):
    return x * F.sigmoid(x)

class FeatureExtractor(nn.Module):
    def __init__(self, cnn, feature_layer=11):
        super(FeatureExtractor, self).__init__()
        self.features = nn.Sequential(*list(cnn.features.children())[:(feature_layer+1)])

    def forward(self, x):
        return self.features(x)


class residualBlock(nn.Module):
    def __init__(self, in_channels=64, k=3, n=64, s=1):
        super(residualBlock, self).__init__()

        self.conv1 = nn.Conv2d(in_channels, n, k, stride=s, padding=1)
        self.bn1 = nn.BatchNorm2d(n)
        self.conv2 = nn.Conv2d(n, n, k, stride=s, padding=1)
        self.bn2 = nn.BatchNorm2d(n)

    def forward(self, x):
        y = sigmoid_mul(self.bn1(self.conv1(x)))
        return self.bn2(self.conv2(y)) + x

class upsampleBlock(nn.Module):
    # Implements resize-convolution
    def __init__(self, in_channels, out_channels):
        super(upsampleBlock, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, 3, stride=1, padding=1)
        self.shuffler = nn.PixelShuffle(2)

    def forward(self, x):
        return sigmoid_mul(self.shuffler(self.conv(x)))


### Define the network

In [0]:
class Generator(nn.Module):
    def __init__(self, n_residual_blocks, upsample_factor):
        super(Generator, self).__init__()
        self.n_residual_blocks = n_residual_blocks
        self.upsample_factor = upsample_factor

        self.conv1 = nn.Conv2d(3, 64, 9, stride=1, padding=4)

        for i in range(self.n_residual_blocks):
            self.add_module('residual_block' + str(i+1), residualBlock())

        self.conv2 = nn.Conv2d(64, 64, 3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(64)

        for i in range(int(self.upsample_factor/2)):
            self.add_module('upsample' + str(i+1), upsampleBlock(64, 256))

        self.conv3 = nn.Conv2d(64, 3, 9, stride=1, padding=4)

    def forward(self, x):
        x = sigmoid_mul(self.conv1(x))

        y = x.clone()
        for i in range(self.n_residual_blocks):
            y = self.__getattr__('residual_block' + str(i+1))(y)

        x = self.bn2(self.conv2(y)) + x

        for i in range(int(self.upsample_factor/2)):
            x = self.__getattr__('upsample' + str(i+1))(x)

        return self.conv3(x)

In [0]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, 3, stride=1, padding=1)

        self.conv2 = nn.Conv2d(64, 64, 3, stride=2, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, 3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.conv4 = nn.Conv2d(128, 128, 3, stride=2, padding=1)
        self.bn4 = nn.BatchNorm2d(128)
        self.conv5 = nn.Conv2d(128, 256, 3, stride=1, padding=1)
        self.bn5 = nn.BatchNorm2d(256)
        self.conv6 = nn.Conv2d(256, 256, 3, stride=2, padding=1)
        self.bn6 = nn.BatchNorm2d(256)
        self.conv7 = nn.Conv2d(256, 512, 3, stride=1, padding=1)
        self.bn7 = nn.BatchNorm2d(512)
        self.conv8 = nn.Conv2d(512, 512, 3, stride=2, padding=1)
        self.bn8 = nn.BatchNorm2d(512)

        # Replaced original paper FC layers with FCN
        self.conv9 = nn.Conv2d(512, 1, 1, stride=1, padding=1)

    def forward(self, x):
        x = sigmoid_mul(self.conv1(x))

        x = sigmoid_mul(self.bn2(self.conv2(x)))
        x = sigmoid_mul(self.bn3(self.conv3(x)))
        x = sigmoid_mul(self.bn4(self.conv4(x)))
        x = sigmoid_mul(self.bn5(self.conv5(x)))
        x = sigmoid_mul(self.bn6(self.conv6(x)))
        x = sigmoid_mul(self.bn7(self.conv7(x)))
        x = sigmoid_mul(self.bn8(self.conv8(x)))

        x = self.conv9(x)
        return F.sigmoid(F.avg_pool2d(x, x.size()[2:])).view(x.size()[0], -1)

In [0]:
def train():
  dataloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE,
                                           shuffle=True, num_workers=2)

  generator = Generator(16, 2)
#   print(generator)

  discriminator = Discriminator()

#   print(discriminator)

  # For the content loss
  feature_extractor = FeatureExtractor(torchvision.models.vgg19(pretrained=True))
#   print(feature_extractor)
  content_criterion = nn.MSELoss()
  adversarial_criterion = nn.BCELoss()

  ones_const = Variable(torch.ones(BATCH_SIZE, 1))

  # if gpu is to be used
  if CUDA:
      generator.cuda()
      discriminator.cuda()
      feature_extractor.cuda()
      content_criterion.cuda()
      adversarial_criterion.cuda()
      ones_const = ones_const.cuda()

  optim_generator = optim.Adam(generator.parameters(), lr=0.0001)
  optim_discriminator = optim.Adam(discriminator.parameters(), lr=0.0001)

  configure('./logs/' + STL10 + '-' + str(BATCH_SIZE) + '-' + str(0.0001) + '-' + str(0.0001), flush_secs=5)
  visualizer = Visualizer(image_size=HEIGHT*2)

  low_res = torch.FloatTensor(BATCH_SIZE, 3, HEIGHT, WIDTH)

  # Pre-train generator using raw MSE loss
  print('Generator pre-training')
  for epoch in range(2):
      mean_generator_content_loss = 0.0

      for i, data in enumerate(dataloader):
          # Generate data
          high_res_real, _ = data

          # Downsample images to low resolution
          for j in range(BATCH_SIZE):
              low_res[j] = scale(high_res_real[j])
              high_res_real[j] = normalize(high_res_real[j])

          # Generate real and fake inputs
          if CUDA:
              high_res_real = Variable(high_res_real.cuda())
              high_res_fake = generator(Variable(low_res).cuda())
          else:
              high_res_real = Variable(high_res_real)
              high_res_fake = generator(Variable(low_res))

          ######### Train generator #########
          generator.zero_grad()

          generator_content_loss = content_criterion(high_res_fake, high_res_real)
          mean_generator_content_loss += generator_content_loss.data[0]

          generator_content_loss.backward()
          optim_generator.step()

          ######### Status and display #########
          print('\r[%d/%d][%d/%d] Generator_MSE_Loss: %.4f' % (epoch, 2, i, len(dataloader), generator_content_loss.data[0]))
          visualizer.show(low_res, high_res_real.cpu().data, high_res_fake.cpu().data)

      print('\r[%d/%d][%d/%d] Generator_MSE_Loss: %.4f\n' % (epoch, 2, i, len(dataloader), mean_generator_content_loss/len(dataloader)))
      log_value('generator_mse_loss', mean_generator_content_loss/len(dataloader), epoch)

  #  training
  optim_generator = optim.Adam(generator.parameters(), lr=0.0001*0.1)
  optim_discriminator = optim.Adam(discriminator.parameters(), lr=0.0001*0.1)

  print('SRGAN training')
  for epoch in range(EPOCHS):
      mean_generator_content_loss = 0.0
      mean_generator_adversarial_loss = 0.0
      mean_generator_total_loss = 0.0
      mean_discriminator_loss = 0.0

      for i, data in enumerate(dataloader):
          # Generate data
          high_res_real, _ = data

          # Downsample images to low resolution
          for j in range(BATCH_SIZE):
              low_res[j] = scale(high_res_real[j])
              high_res_real[j] = normalize(high_res_real[j])

          # Generate real and fake inputs
          if CUDA:
              high_res_real = Variable(high_res_real.cuda())
              high_res_fake = generator(Variable(low_res).cuda())
              target_real = Variable(torch.rand(BATCH_SIZE,1)*0.5 + 0.7).cuda()
              target_fake = Variable(torch.rand(BATCH_SIZE,1)*0.3).cuda()
          else:
              high_res_real = Variable(high_res_real)
              high_res_fake = generator(Variable(low_res))
              target_real = Variable(torch.rand(BATCH_SIZE,1)*0.5 + 0.7)
              target_fake = Variable(torch.rand(BATCH_SIZE,1)*0.3)

          ######### Train discriminator #########
          discriminator.zero_grad()

          discriminator_loss = adversarial_criterion(discriminator(high_res_real), target_real) + \
                               adversarial_criterion(discriminator(Variable(high_res_fake.data)), target_fake)
          mean_discriminator_loss += discriminator_loss.data[0]

          discriminator_loss.backward()
          optim_discriminator.step()

          ######### Train generator #########
          generator.zero_grad()

          real_features = Variable(feature_extractor(high_res_real).data)
          fake_features = feature_extractor(high_res_fake)

          generator_content_loss = content_criterion(high_res_fake, high_res_real) + 0.006*content_criterion(fake_features, real_features)
          mean_generator_content_loss += generator_content_loss.data[0]
          generator_adversarial_loss = adversarial_criterion(discriminator(high_res_fake), ones_const)
          mean_generator_adversarial_loss += generator_adversarial_loss.data[0]

          generator_total_loss = generator_content_loss + 1e-3*generator_adversarial_loss
          mean_generator_total_loss += generator_total_loss.data[0]

          generator_total_loss.backward()
          optim_generator.step()   

          ######### Status and display #########
          print('\r[%d/%d][%d/%d] Discriminator_Loss: %.4f Generator_Loss (Content/Advers/Total): %.4f/%.4f/%.4f' % (epoch, EPOCHS, i, len(dataloader),
          discriminator_loss.data[0], generator_content_loss.data[0], generator_adversarial_loss.data[0], generator_total_loss.data[0]))
          visualizer.show(low_res, high_res_real.cpu().data, high_res_fake.cpu().data)

      print('\r[%d/%d][%d/%d] Discriminator_Loss: %.4f Generator_Loss (Content/Advers/Total): %.4f/%.4f/%.4f\n' % (epoch, EPOCHS, i, len(dataloader),
      mean_discriminator_loss/len(dataloader), mean_generator_content_loss/len(dataloader), 
      mean_generator_adversarial_loss/len(dataloader), mean_generator_total_loss/len(dataloader)))

      log_value('generator_content_loss', mean_generator_content_loss/len(dataloader), epoch)
      log_value('generator_adversarial_loss', mean_generator_adversarial_loss/len(dataloader), epoch)
      log_value('generator_total_loss', mean_generator_total_loss/len(dataloader), epoch)
      log_value('discriminator_loss', mean_discriminator_loss/len(dataloader), epoch)

In [25]:
transform = transforms.Compose([transforms.RandomCrop(LR_IMAGE_SIZE*UPSCALING_FACTOR),
                                transforms.ToTensor()])

normalize = transforms.Normalize(mean = [0.485, 0.456, 0.406],
                                std = [0.229, 0.224, 0.225])

scale = transforms.Compose([transforms.ToPILImage(),
                            transforms.Scale(LR_IMAGE_SIZE),
                            transforms.ToTensor(),
                            transforms.Normalize(mean = [0.485, 0.456, 0.406],
                                                std = [0.229, 0.224, 0.225])
                            ])

# Equivalent to un-normalizing ImageNet (for correct visualization)
unnormalize = transforms.Normalize(mean = [-2.118, -2.036, -1.804], std = [4.367, 4.464, 4.444])


dataloader = torch.utils.data.DataLoader(testset, batch_size=BATCH_SIZE,
                                         shuffle=False, num_workers=2)

generator = Generator(16, int(UPSCALING_FACTOR))

# print(generator)

discriminator = Discriminator()

# print(discriminator)

# For the content loss
feature_extractor = FeatureExtractor(torchvision.models.vgg19(pretrained=True))
print(feature_extractor)
content_criterion = nn.MSELoss()
adversarial_criterion = nn.BCELoss()

target_real = Variable(torch.ones(BATCH_SIZE,1))
target_fake = Variable(torch.zeros(BATCH_SIZE,1))

# if gpu is to be used
if CUDA:
    generator.cuda()
    discriminator.cuda()
    feature_extractor.cuda()
    content_criterion.cuda()
    adversarial_criterion.cuda()
    target_real = target_real.cuda()
    target_fake = target_fake.cuda()

low_res = torch.FloatTensor(BATCH_SIZE, 3, LR_IMAGE_SIZE, LR_IMAGE_SIZE)

print('Test started...')
mean_generator_content_loss = 0.0
mean_generator_adversarial_loss = 0.0
mean_generator_total_loss = 0.0
mean_discriminator_loss = 0.0

# Set evaluation mode (not training)
generator.eval()
discriminator.eval()

for i, data in enumerate(dataloader):
    # Generate data
    high_res_real, _ = data

    # Downsample images to low resolution
    for j in range(BATCH_SIZE):
        low_res[j] = scale(high_res_real[j])
        high_res_real[j] = normalize(high_res_real[j])

    # Generate real and fake inputs
    if CUDA:
        high_res_real = Variable(high_res_real.cuda())
        high_res_fake = generator(Variable(low_res).cuda())
    else:
        high_res_real = Variable(high_res_real)
        high_res_fake = generator(Variable(low_res))
    
    ######### Test discriminator #########

    discriminator_loss = adversarial_criterion(discriminator(high_res_real), target_real) + \
                            adversarial_criterion(discriminator(Variable(high_res_fake.data)), target_fake)
    mean_discriminator_loss += discriminator_loss.data[0]

    ######### Test generator #########

    real_features = Variable(feature_extractor(high_res_real).data)
    fake_features = feature_extractor(high_res_fake)

    generator_content_loss = content_criterion(high_res_fake, high_res_real) + 0.006*content_criterion(fake_features, real_features)
    mean_generator_content_loss += generator_content_loss.data[0]
    generator_adversarial_loss = adversarial_criterion(discriminator(high_res_fake), target_real)
    mean_generator_adversarial_loss += generator_adversarial_loss.data[0]

    generator_total_loss = generator_content_loss + 1e-3*generator_adversarial_loss
    mean_generator_total_loss += generator_total_loss.data[0]
    print("i :"+str(i))
    ######### Status and display #########
    print('\r[%d/%d] Discriminator_Loss: %.4f Generator_Loss (Content/Advers/Total): %.4f/%.4f/%.4f' % (i, len(dataloader),
    discriminator_loss.data[0], generator_content_loss.data[0], generator_adversarial_loss.data[0], generator_total_loss.data[0]))

    for j in range(BATCH_SIZE):
        save_image(unnormalize(high_res_real.data[j]), 'output/high_res_real/' + str(i*BATCH_SIZE + j) + '.png')
        save_image(unnormalize(high_res_fake.data[j]), 'output/high_res_fake/' + str(i*BATCH_SIZE + j) + '.png')
        save_image(unnormalize(low_res[j]), 'output/low_res/' + str(i*BATCH_SIZE + j) + '.png')

print ("out of all loops")
print('\r[%d/%d] Discriminator_Loss: %.4f Generator_Loss (Content/Advers/Total): %.4f/%.4f/%.4f\n' % (i, len(dataloader),
mean_discriminator_loss/len(dataloader), mean_generator_content_loss/len(dataloader), 
mean_generator_adversarial_loss/len(dataloader), mean_generator_total_loss/len(dataloader)))

/usr/local/lib/python3.6/dist-packages/torchvision/transforms/transforms.py:188: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  "please use transforms.Resize instead.")


FeatureExtractor(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
  )
)
Test started...


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1006: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:80: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:88: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:90: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:93: UserWarning: invalid index of a 0-dim tensor. This will be an error in 

i :0
[0/1000] Discriminator_Loss: 1.3863 Generator_Loss (Content/Advers/Total): 9.8345/0.6900/9.8352
i :1
[1/1000] Discriminator_Loss: 1.3863 Generator_Loss (Content/Advers/Total): 13.7037/0.6900/13.7044
i :2
[2/1000] Discriminator_Loss: 1.3863 Generator_Loss (Content/Advers/Total): 11.9553/0.6900/11.9560
i :3
[3/1000] Discriminator_Loss: 1.3863 Generator_Loss (Content/Advers/Total): 12.9347/0.6900/12.9353
i :4
[4/1000] Discriminator_Loss: 1.3863 Generator_Loss (Content/Advers/Total): 13.9657/0.6900/13.9664
i :5
[5/1000] Discriminator_Loss: 1.3863 Generator_Loss (Content/Advers/Total): 9.5029/0.6900/9.5036
i :6
[6/1000] Discriminator_Loss: 1.3863 Generator_Loss (Content/Advers/Total): 13.4157/0.6900/13.4164
i :7
[7/1000] Discriminator_Loss: 1.3863 Generator_Loss (Content/Advers/Total): 10.9856/0.6900/10.9863
i :8
[8/1000] Discriminator_Loss: 1.3863 Generator_Loss (Content/Advers/Total): 11.9293/0.6900/11.9300
i :9
[9/1000] Discriminator_Loss: 1.3863 Generator_Loss (Content/Advers/Total

Process Process-10:
Process Process-9:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 96, in _worker_loop
    r = index_queue.get(timeout=MANAGER_STATUS_CHECK_INTERVAL)
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 104, in get
    if not self._poll(timeout):
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 257, in poll
    return self._poll(timeout)
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 

i :84
[84/1000] Discriminator_Loss: 1.3863 Generator_Loss (Content/Advers/Total): 13.3432/0.6900/13.3439


  File "/usr/lib/python3.6/multiprocessing/connection.py", line 257, in poll
    return self._poll(timeout)


KeyboardInterrupt: ignored

  File "/usr/lib/python3.6/multiprocessing/connection.py", line 414, in _poll
    r = wait([self], timeout)
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 911, in wait
    ready = selector.select(timeout)
  File "/usr/lib/python3.6/selectors.py", line 376, in select
    fd_event_list = self._poll.poll(timeout)
KeyboardInterrupt


Reference :https://github.com/eriklindernoren/PyTorch-GAN

In [0]:
# from google.colab import drive
# drive.mount('/content/gdrive') 

In [0]:
# torch.cuda.is_available()
!ls output/low_res

In [0]:
!rm -rf output/*
!mkdir output/low_res
!mkdir output/high_res_fake
!mkdir output/high_res_real